In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

import statsmodels.api as sm
from statsmodels.tsa.api import Holt

from env import get_connection

## Exercises

The end result of this exercise should be a Jupyter notebook named `model`.

Using [saas.csv](https://ds.codeup.com/saas.csv) or log data from API usage

1. Split data (train/test) and resample by any period, except daily, and aggregate using the sum. 

In [2]:
df = pd.read_csv("saas.csv", parse_dates=['Month_Invoiced'], usecols=['Month_Invoiced', 'Subscription_Type', 'Amount'], index_col='Month_Invoiced').sort_index()

In [3]:
#df.set_index('Month_Invoiced', inplace=True).sort_index()

In [4]:
train_index = int(len(df) * 0.7)

In [5]:
train = df[:train_index]

In [6]:
train.shape

(553320, 2)

In [7]:
test = df[train_index:]

In [8]:
test.shape

(237138, 2)

In [9]:
df.shape[0] == train.shape[0] + test.shape[0]

True

In [10]:
train.resample('W').sum()

,Subscription_Type,Amount
Month_Invoiced,,
2014-02-02,7170.0,35850.0
2014-02-09,0.0,0.0
2014-02-16,0.0,0.0
2014-02-23,0.0,0.0
2014-03-02,7438.0,37190.0
...,...,...
2016-08-07,0.0,0.0
2016-08-14,0.0,0.0
2016-08-21,0.0,0.0


2. Forecast, plot and evaluate using each at least 4 of the methods we discussed:
    - Last Observed Value
    - Simple Average
    - Moving Average
    - Holt's Linear Trend 
    - Holt's Seasonal Trend
    - Based on previous year/month/etc., this is up to you.

Bonus: 
1. Using the store item demand data, create a forecast of `sales_total` and `quantity` for 2018 using the `Previous Cycle` approach.  .  
2. Predict 2018 total **monthly** sales for a single store and/or item by creating a model using prophet.
3. Return a dataframe with the month, store_id, y-hat, and the confidence intervals (y-hat lower, y-hat upper).
4. Plot the 2018 monthly sales predictions.